Creating Designs by Leveraging OpenAI and Gradio UI

In [ ]:
## Project Overview

This project demonstrates the development of an AI-powered design generation platform that combines OpenAI's DALL·E image generation capabilities with Gradio's intuitive web interface. Built for a creative agency context, this tool enables rapid conceptualization and prototyping of marketing campaign visuals through natural language descriptions.

**Project Context:** As a designer at a creative agency, I developed this web-based platform to streamline the ideation process for Netflix-style marketing campaign concepts. The tool allows marketing teams and designers to generate banner and poster designs using text prompts, significantly reducing the time from concept to visual mockup.

**Key Technologies:**
- **OpenAI DALL·E 3** - State-of-the-art text-to-image generation
- **Gradio** - Python framework for building ML web interfaces
- **PIL (Pillow)** - Image processing and manipulation
- **Python Requests** - HTTP client for API interactions

**Target Use Case:** Generating promotional poster concepts for streaming platform marketing campaigns (hypothetical Netflix-style concepts)

In [ ]:
# Import required libraries
import gradio as gr  # Web UI framework
import requests  # HTTP client for image downloads
from PIL import Image  # Image processing library
from io import BytesIO  # In-memory bytes buffer
from openai import OpenAI  # OpenAI official client library
import os  # Environment variable access
import time  # For potential retry logic

# Display package versions for reproducibility
print("Environment Setup Complete")
print(f"Gradio version: {gr.__version__}")
print(f"OpenAI client library imported successfully")
print("-" * 50)

Environment Setup Complete
Gradio version: 5.50.0
OpenAI client library imported successfully
--------------------------------------------------


In [ ]:
# Configure OpenAI API client with secure key loading

def load_api_key():
    """
    Load OpenAI API key from environment with fallback support.

    Returns:
        str: OpenAI API key

    Raises:
        ValueError: If no API key found in any source
    """
    # Try Google Colab secrets first
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        print("✓ API key loaded from Google Colab secrets")
        return api_key
    except:
        pass

    # Try environment variable
    api_key = os.environ.get('OPENAI_API_KEY')
    if api_key:
        print("✓ API key loaded from environment variable")
        return api_key

    # No key found
    raise ValueError(
        "OpenAI API key not found. Please set OPENAI_API_KEY in:\n"
        "  - Google Colab: Secrets (key icon in sidebar)\n"
        "  - Local: export OPENAI_API_KEY='your-key-here'"
    )

# Initialize OpenAI client
try:
    api_key = load_api_key()
    client = OpenAI(api_key=api_key)
    print("✓ OpenAI client initialized successfully")
    print("-" * 50)
except Exception as e:
    print(f"✗ Error initializing OpenAI client: {e}")
    raise

✓ API key loaded from Google Colab secrets
✓ OpenAI client initialized successfully
--------------------------------------------------


In [ ]:
def generate_image(prompt: str) -> Image.Image:
    """
    Generate a marketing poster image using OpenAI DALL·E 3.

    This function handles the complete image generation pipeline:
    - Validates user input
    - Calls OpenAI API with optimized parameters for poster design
    - Downloads and processes the generated image
    - Provides detailed error feedback

    Args:
        prompt (str): Text description of the desired poster design

    Returns:
        PIL.Image.Image: Generated poster image (1024x1792 portrait format)

    Raises:
        ValueError: If prompt is empty or invalid
        Exception: If API call fails or image cannot be processed
    """

    # Input validation
    if not prompt or not prompt.strip():
        raise ValueError("Please enter a prompt description for your poster design.")

    if len(prompt.strip()) < 10:
        raise ValueError("Please provide a more detailed prompt (at least 10 characters).")

    print(f"Generating image for prompt: '{prompt[:100]}...'")

    try:
        # Call OpenAI DALL·E 3 API
        response = client.images.generate(
            model="dall-e-3",  # Latest DALL·E model with best prompt adherence
            prompt=prompt,
            size="1024x1792",  # Portrait format optimized for posters
            quality="standard",  # Options: "standard" or "hd" (hd costs more)
            n=1,  # Number of images to generate
        )

        # Extract image URL from response
        image_url = response.data[0].url
        print(f"✓ Image generated successfully")
        print(f"  URL: {image_url[:80]}...")

        # Download image from URL
        print("  Downloading image...")
        image_response = requests.get(image_url, timeout=30)
        image_response.raise_for_status()  # Raise exception for HTTP errors

        # Convert bytes to PIL Image
        image_bytes = BytesIO(image_response.content)
        image = Image.open(image_bytes)

        print(f"✓ Image processed successfully (Size: {image.size})")
        print("-" * 50)

        return image

    except Exception as e:
        # Detailed error handling for different failure modes
        error_message = str(e)

        if "content_policy_violation" in error_message.lower():
            raise Exception(
                "Content Policy Violation: The prompt was flagged by OpenAI's safety system. "
                "Please revise to remove any potentially sensitive content."
            )
        elif "rate_limit" in error_message.lower():
            raise Exception(
                "Rate Limit Exceeded: Too many requests. Please wait a moment and try again."
            )
        elif "insufficient_quota" in error_message.lower():
            raise Exception(
                "API Quota Exceeded: Your OpenAI account has insufficient credits. "
                "Please add credits at https://platform.openai.com/account/billing"
            )
        elif "invalid_api_key" in error_message.lower():
            raise Exception(
                "Invalid API Key: Please check your OPENAI_API_KEY configuration."
            )
        else:
            # Generic error with full details for debugging
            raise Exception(f"Image generation failed: {error_message}")

# Test the function (optional - comment out for production)
# test_image = generate_image("A minimalist movie poster with bold typography")
# print(f"Test successful: Generated image of size {test_image.size}")

In [ ]:
# Create Gradio web interface

# Define example prompts that demonstrate effective prompt engineering
example_prompts = [
    [
        "A cinematic movie poster for a sci-fi thriller series, featuring a lone astronaut "
        "silhouetted against a massive alien planet with rings, dark space background with "
        "nebula colors of deep purple and blue, mysterious and epic atmosphere, "
        "professional streaming platform style, bold minimalist composition"
    ],
    [
        "A vibrant animated series poster in the style of modern 3D animation, "
        "featuring a diverse group of teenage heroes in dynamic action poses, "
        "colorful urban fantasy setting with floating islands and magical portals, "
        "energetic and adventurous mood, bright saturated colors with neon accents, "
        "perfect for young adult audience"
    ],
    [
        "A dark psychological horror series poster with film noir aesthetic, "
        "featuring a mysterious Victorian mansion at night with a single lit window, "
        "fog and shadows, ominous red and black color scheme, vintage typography style, "
        "suspenseful and eerie atmosphere, inspired by gothic literature"
    ],
    [
        "A romantic comedy series poster with warm pastel color palette, "
        "featuring silhouettes of two people under string lights in a cozy café setting, "
        "soft bokeh background, cheerful and heartwarming mood, "
        "elegant modern design with hand-drawn elements, indie film aesthetic"
    ],
    [
        "An epic fantasy series poster with painterly art style, "
        "featuring a medieval warrior queen holding a glowing sword, "
        "dramatic storm clouds and castle in background, rich jewel tones of emerald and gold, "
        "cinematic lighting with god rays, heroic and legendary atmosphere"
    ],
    [
        "A documentary series poster with photorealistic style, "
        "featuring Earth from space with visible city lights at night, "
        "inspiring and educational mood, clean modern typography, "
        "professional and sophisticated design with teal and orange color grading"
    ]
]

# Create Gradio interface using gr.Interface
demo = gr.Interface(
    fn=generate_image,  # Function to call when user submits

    inputs=gr.Textbox(
        label="Campaign Concept Description",
        placeholder="Describe your poster design in detail: genre, mood, visual elements, color palette, style...",
        lines=5,  # Multi-line input for detailed prompts
        max_lines=10
    ),

    outputs=gr.Image(
        label="Generated Poster Design",
        type="pil",  # Return PIL Image object
        show_label=True
    ),

    title="🎬 AI-Powered Marketing Poster Generator",

    description="""
    Generate professional poster concepts for streaming platform marketing campaigns using AI.

    **How to use:**
    1. Describe your ideal poster in the text box (be specific about style, mood, colors, composition)
    2. Click Submit and wait ~20-30 seconds for generation
    3. Download the result using the download button below the image

    **Tips for better results:**
    - Include genre/style keywords (sci-fi, romantic comedy, documentary, etc.)
    - Specify mood/atmosphere (dark, cheerful, mysterious, epic)
    - Describe visual elements (characters, settings, objects)
    - Mention color palette preferences
    - Reference composition style (minimalist, busy, centered, etc.)

    *Note: All generated images are original AI creations and do not reproduce copyrighted content.*
    """,

    examples=example_prompts,  # Clickable example buttons

    article="""
    ### About This Tool

    This platform uses OpenAI's DALL·E 3 model to transform text descriptions into high-quality poster designs.
    Perfect for rapid concept exploration, client pitches, and creative brainstorming sessions.

    **Workflow Integration:**
    - Generate 10-15 concepts in a single brainstorming session
    - Use outputs as references for designers or send directly to clients
    - Iterate quickly by refining prompts based on feedback

    **Developed for creative agencies by designers, for designers.**
    """,

    theme=gr.themes.Soft(),  # Clean, professional theme

    allow_flagging="never",  # Disable flagging feature for production use
)

print("✓ Gradio interface created successfully")
print("  Ready to launch!")

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


✓ Gradio interface created successfully
  Ready to launch!


In [ ]:
# Create Gradio web interface

# Define example prompts that demonstrate effective prompt engineering
example_prompts = [
    [
        "A cinematic movie poster for a sci-fi thriller series, featuring a lone astronaut "
        "silhouetted against a massive alien planet with rings, dark space background with "
        "nebula colors of deep purple and blue, mysterious and epic atmosphere, "
        "professional streaming platform style, bold minimalist composition"
    ],
    [
        "A vibrant animated series poster in the style of modern 3D animation, "
        "featuring a diverse group of teenage heroes in dynamic action poses, "
        "colorful urban fantasy setting with floating islands and magical portals, "
        "energetic and adventurous mood, bright saturated colors with neon accents, "
        "perfect for young adult audience"
    ],
    [
        "A dark psychological horror series poster with film noir aesthetic, "
        "featuring a mysterious Victorian mansion at night with a single lit window, "
        "fog and shadows, ominous red and black color scheme, vintage typography style, "
        "suspenseful and eerie atmosphere, inspired by gothic literature"
    ],
    [
        "A romantic comedy series poster with warm pastel color palette, "
        "featuring silhouettes of two people under string lights in a cozy café setting, "
        "soft bokeh background, cheerful and heartwarming mood, "
        "elegant modern design with hand-drawn elements, indie film aesthetic"
    ],
    [
        "An epic fantasy series poster with painterly art style, "
        "featuring a medieval warrior queen holding a glowing sword, "
        "dramatic storm clouds and castle in background, rich jewel tones of emerald and gold, "
        "cinematic lighting with god rays, heroic and legendary atmosphere"
    ],
    [
        "A documentary series poster with photorealistic style, "
        "featuring Earth from space with visible city lights at night, "
        "inspiring and educational mood, clean modern typography, "
        "professional and sophisticated design with teal and orange color grading"
    ]
]

# Create Gradio interface using gr.Interface
demo = gr.Interface(
    fn=generate_image,  # Function to call when user submits

    inputs=gr.Textbox(
        label="Campaign Concept Description",
        placeholder="Describe your poster design in detail: genre, mood, visual elements, color palette, style...",
        lines=5,  # Multi-line input for detailed prompts
        max_lines=10
    ),

    outputs=gr.Image(
        label="Generated Poster Design",
        type="pil",  # Return PIL Image object
        show_label=True
    ),

    title="🎬 AI-Powered Marketing Poster Generator",

    description="""
    Generate professional poster concepts for streaming platform marketing campaigns using AI.

    **How to use:**
    1. Describe your ideal poster in the text box (be specific about style, mood, colors, composition)
    2. Click Submit and wait ~20-30 seconds for generation
    3. Download the result using the download button below the image

    **Tips for better results:**
    - Include genre/style keywords (sci-fi, romantic comedy, documentary, etc.)
    - Specify mood/atmosphere (dark, cheerful, mysterious, epic)
    - Describe visual elements (characters, settings, objects)
    - Mention color palette preferences
    - Reference composition style (minimalist, busy, centered, etc.)

    *Note: All generated images are original AI creations and do not reproduce copyrighted content.*
    """,

    examples=example_prompts,  # Clickable example buttons

    article="""
    ### About This Tool

    This platform uses OpenAI's DALL·E 3 model to transform text descriptions into high-quality poster designs.
    Perfect for rapid concept exploration, client pitches, and creative brainstorming sessions.

    **Workflow Integration:**
    - Generate 10-15 concepts in a single brainstorming session
    - Use outputs as references for designers or send directly to clients
    - Iterate quickly by refining prompts based on feedback

    **Developed for creative agencies by designers, for designers.**
    """,

    theme=gr.themes.Soft(),  # Clean, professional theme

    allow_flagging="never",  # Disable flagging feature for production use
)

print("✓ Gradio interface created successfully")
print("  Ready to launch!")

✓ Gradio interface created successfully
  Ready to launch!


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
# Launch the Gradio web application

# Launch with share=True to generate public URL for remote access
# Change to share=False for local-only access (more secure)
demo.launch(
    share=True,  # Creates temporary public URL (expires in 72 hours)
    debug=False,  # Set to True for detailed error logging
    show_error=True,  # Display errors in UI (useful for users)
)

# The interface is now live!
# - Click the local URL to open in your browser
# - Share the public URL with team members (if share=True)
# - Stop the server by interrupting the kernel

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15c896abf8c7444b78.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
